# ML в Биологии
## 9. Unsupervised Learning

In [ ]:
import numpy                as np
import matplotlib.pyplot    as plt
import matplotlib.cm        as cm
import pandas               as pd
import ipywidgets           as widgets
from sklearn.decomposition  import PCA
import seaborn              as sns
import numpy                as np
from tqdm.notebook          import tqdm

from sklearn.cluster import \
    KMeans, \
    AgglomerativeClustering, \
    DBSCAN

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

### Задача 1

Рассмотрим датасет [**Leaf Classification**](https://www.kaggle.com/c/leaf-classification).

Данные содержат 1584 изображений образцов листьев (16 изображений для 99 видов). Размер некоторых изображений изменен, в результате чего все изображения имеют одинаковый размер $170×250$.



1. Скачайте данные с сайта.

2. Загрузим все изображения с помощью `plt.imread` и визуализируем некоторые из них. Каждое изображение — матрица размера $170×250$.

In [ ]:
!unzip /content/scaled_images.zip

In [ ]:
n_images = 1584
im_size = (170, 250)

X = []
for i in tqdm(range(n_images)):
    new_img = plt.imread('scaled_images/' + str(i + 1) + '.jpg')
    X.append(new_img)

X = np.array(X)

plt.figure(figsize=(10, 10))
for i in range(9):
    plt.subplot(3, 3, i + 1)
    plt.imshow(X[i], cmap='gray')
    plt.axis('off')
plt.show()

Сделаем также так, чтобы каждое изображение было представлено не матрицей, а одним вектором из всех пикселей

In [ ]:
X = X.reshape(len(X), -1)
assert(X.shape ==(1584, 42500))

3. В файле `train_labels.csv` указаны номера образцов листьев, которые относятся к обучающей части данных, а также их виды.

In [ ]:
labels = pd.read_csv('train_labels.csv')
labels.head(20)

Разделите данные на обучающую и тестовую часть.

In [ ]:
train_ids = np.array(labels.id)
test_ids = np.array(list(set(np.arange(1, len(X) + 1)) - set(train_ids)))

X_train, X_test = X[train_ids - 1], X[test_ids - 1]
y_train = labels

Отсортируем теперь по алфавиту названия видов и построим отображение строки в индекс.

In [ ]:
species_names = sorted(np.unique(labels.species))
name_to_ind = dict([(name, i) for (i, name) in enumerate(species_names)])
labels.species = labels.species.map(name_to_ind)

4. На обучающей части данных постройте 30 главных компонент. Какую долю дисперсии данных они объясняют? Какую долю дисперсии объясняет каждая компонента отдельно? Постройте график доли объясненной дисперсии (зависимость доли от номера компоненты). Сделайте вывод.

    * **Пояснение**: доля объясненной дисперсии - это показатель, характеризующий какую долю от общей дисперсии в данных объясняет данная компонента. Почитайте в документации метода главных компонент о [`explained_variance_ratio_`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html#:~:text=in%20version%200.18.-,explained_variance_ratio_,-ndarray%20of%20shape).*

In [ ]:
from sklearn.preprocessing import StandardScaler
pca = PCA(30)
X_pca = pca.fit_transform(X_train)

In [ ]:
plt.figure(figsize=(7, 5))

plt.plot(np.arange(1, 31), np.cumsum(pca.explained_variance_ratio_))

plt.title("Суммарная доля объяснённой дисперсии")
plt.xlabel("Number of components")
plt.ylabel("Cumulative explained_variance_ratio")

plt.grid()

np.cumsum(pca.explained_variance_ratio_)[-1]

In [ ]:
plt.figure(figsize=(7, 5))

plt.plot(np.arange(1, 31), pca.explained_variance_ratio_)

plt.title("Доля объяснённой дисперсии каждой компоненты")
plt.xlabel("Number of component")
plt.ylabel("Explained_variance_ratio")

plt.grid()

**Вывод:**

Видим, что суммарно первые 30 компонент описывают примерно 80% всей дисперсии.

Наибольший вклад вносит первая компонента. После 10-й компоненты вклад каждой новой стремится к нулю.

5. Визуализируйте главные компоненты: покажите, какие картинки из себя представляют главные компоненты. Для этого перейдите обратно из представления изображения в виде одного длинного вектора к матрице. Можете ли вы их как-то охарактеризовать?

In [ ]:
fig, axs = plt.subplots(figsize=(20, 20), ncols=5, nrows=6)
for i in range(6):
    for j in range(5):
        axs[i][j].imshow(pca.components_[5*i + j].reshape(170, 250))
        axs[i][j].set_title(f"Component №{5*i + j + 1}")

**Вывод:**

Для каждой компоненты видно некое усреднение всех форм листа с преобладанием одной из форм: какие-то горизонтально вытянуты, какие-то вертикально, какие-то звёздчатой формы.

6. Визуализируйте обучающую часть данных в проекции на две первых главных компоненты. Цвет точки должен соответствовать виду образца. Используйте `cmap=’Set1’` во избежание градации цвета по номеру вида. Наблюдаются ли какие-либо закономерности?

In [ ]:
pc1, pc2 = X_pca[:, 0], X_pca[:, 1]
plt.scatter(pc1, pc2, c=labels.species, cmap='Set1')

**Вывод:**

Какие-то закономерности видны, но сомнительно...

Красные точечки явно примерно на одной линии, остальные тоже образуют условно связные области. Но никаких красивых кластеров.

7. По проекциям данных на первые 30 главных компонент обучите многоклассовую классификацию.

    Используйте любую модель классификации, рассмотренную на нашем курсе.

    Разделите данные на тренировочную и валидационную выборки и проверьте качество этой модели по метрике accuracy. Так как метки классов известны только для части данных, используйте только их. Сравните с результатом без применения PCA. Сделайте выводы.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score as accuracy
from sklearn.ensemble import RandomForestClassifier

In [ ]:
train_ftrs, test_ftrs, train_lbls, test_lbls = train_test_split(X_pca, labels.species, test_size=0.25)

lr = RandomForestClassifier()
lr.fit(train_ftrs, train_lbls)
preds = lr.predict(test_ftrs)

print("Accuracy with PCA: ", accuracy(test_lbls, preds))

In [ ]:
scaler = StandardScaler()

train_ftrs, test_ftrs, train_lbls, test_lbls = train_test_split(X_train, labels.species, test_size=0.25)

X_s = scaler.fit_transform(train_ftrs)
X_ts = scaler.transform(test_ftrs)
X_s.shape

In [ ]:
lr = RandomForestClassifier()
lr.fit(X_s, train_lbls)
preds = lr.predict(X_ts)

print("Accuracy without PCA: ",accuracy(test_lbls, preds))

**Вывод:**

Когда у нас размерность 42500, PCA явно поможет. И он помог. Метрика модели на главных компонентах выше, чем просто на исходном датасете.

In [ ]:
labels

### Задача 2

В кишечнике человека живут триллионы микроорганизмов. Они могут влиять на совершенно различные процессы в организме. Однако на данный момент мы находимся лишь в самом начале пути изучения  механизмов их взаимодействия с организмом хозяина. Сейчас влияние микробиоты изучают во многих сферах медицины: в онкологии, психиатрии...

Состави микробиоты у каждого человека свой. Однако важна не только похожесть состава, но и функции тех или иных организмов, в частности белки, которые они производят. Весьма много информации могут нам дать данные [метагеномного секвенирования](https://www.news-medical.net/life-sciences/Shotgun-Metagenomic-Sequencing.aspx). Этот тип секвенирования позволяет получать короткие последовательности ДНК от всех организмов в образце вместе, то есть не разделяя ДНК по организмам. Использование метагеномного секвнирования помогло обнаружить, насколько распространены гены антибиотиков в наших кишечных бактериях.

На данный момент мы остановимся лишь на изучении того, какие организмы живут в кишечнике различных людей, а не на их функциях. Эти данные можно получить из данных метагеномного секвенирования, рассмотрев различные последовательности ДНК, соответствующие одной из субъединиц рибосомы. Оказывается, что по эти последовательности уникальны у каждого вида, и таким образом легко понять, какие микроорганизмы присутствуют в образце.

Скачайте [датасет](https://www.kaggle.com/antaresnyc/human-metagenomics?select=abundance_stoolsubset.csv). Мы будем использовать файл `abundance_stoolsubset.csv`, в котором лежат уже предобработанные данные о том, ДНК каких организмов обнаружена в образцах микробиоты кишечника и в каком количестве.

Данные были взяты из статьи
["Machine Learning Meta-analysis of Large Metagenomic Datasets: Tools and Biological Insights"
](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1004977) Pasolli et al.


In [ ]:
!unzip /content/archive.zip

In [ ]:
data = pd.read_csv("abundance_stoolsubset.csv")
data.head()

В данных есть много метаданных об образце. Их мы использовать не будем, так как хотим сравнивать людей по микроорганизмам.

Метаданные занимают первые 210 колонок. Удалите их.



In [ ]:
data_clean = data.iloc[:, 210:]
data_clean.head()

Если посмотреть на данные, то можно заметить, что учитываются как количества ДНК организмов каждого вида, так и более высших иерархических единиц. Таким образом, в данных присутствует избыточность, так как некоторые столбцы просто являются линейной комбинацией друг друга.

Снизьте размерность с помощью PCA. Подберите оптимальное количество главных компонент.

In [ ]:
pca = PCA()
X_pca = pca.fit_transform(data_clean)

# Подбираем оптимальное количество главных компонент
explained_variance = pca.explained_variance_ratio_
cumulative_variance = explained_variance.cumsum()

# Находим оптимальное количество компонент, которые объясняют > 90% дисперсии
optimal_components = (cumulative_variance >= 0.9).argmax() + 1
print(f"Оптимальное количество главных компонент: {optimal_components}")

Визуализируйте данные с помощью t-SNE и UMAP. Используйте особенности методов. Не забудьте пояснить свое решение.

In [ ]:
!pip install umap-learn

In [ ]:
from sklearn.manifold import TSNE
from umap.umap_ import UMAP
sns.set_style('white')

X_tsne = TSNE(n_components=2, random_state=42).fit_transform(X_pca[:, :optimal_components])
X_umap = UMAP(n_components=2, random_state=42).fit_transform(X_pca[:, :optimal_components])

In [ ]:
def draw_umap_tsne(figsize, c=None, outliers=np.zeros(X_tsne.shape[0], dtype=bool)):
    """Функция, строящая проекции данных на 2 компоненты, получающиеся в результате применения UMAP

    Args:
        figsize: размер картинки
        c: метки для отображения точек в цвете.
        outliers: выбросы, если таковые имеются
    """
    # t-SNE
    plt.figure(figsize=figsize)
    plt.subplot(1, 2, 1)
    plt.scatter(X_tsne[~outliers, 0], X_tsne[~outliers, 1], c=c, cmap='viridis', s=30)  # Объекты, не являющиеся выбросами
    plt.scatter(X_tsne[outliers, 0], X_tsne[outliers, 1], color='r', s=30, label='Outliers')  # Отдельно рисуем выбросы, если такие есть
    plt.xlabel('Координата 1')
    plt.ylabel('Координата 2')
    plt.title('TSNE: Данные в латентном пространстве')

    # UMAP
    plt.subplot(1, 2, 2)
    plt.scatter(X_umap[~outliers, 0], X_umap[~outliers, 1], c=c, cmap='viridis', s=30)  # Объекты, не являющиеся выбросами
    plt.scatter(X_umap[outliers, 0], X_umap[outliers, 1], color='r', s=300, label='Outliers')  # Отдельно рисуем выбросы, если такие есть
    plt.xlabel('Координата 1')
    plt.ylabel('Координата 2')
    plt.title('UMAP: Данные в латентном пространстве')
    plt.tight_layout()
    plt.show()

In [ ]:
draw_umap_tsne(figsize=(20, 7))

Теперь перейдем к кластеризации. Попробуйте кластеризовать данные в исходном пространстве и в пространстве после применения PCA с помощью DBSCAN, k-means и агломеративной кластеризации. Визуализируйте полученные результаты кластеризации с помощью t-SNE и UMAP, выделяя разные кластеры цветом. В этом может помочь аргумент `c` метода `scatter`. Помимо этого, визуализируйте силуэты точек. Вспомогательный код  для этого можно найти в [документации sklearn](https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html#selecting-the-number-of-clusters-with-silhouette-analysis-on-kmeans-clustering). Хорошо ли работают методы? Попробуйте поизменять  параметры.

Мы понимаем, что это не полноценное исследование, а всего лишь домашнее упражнение. Поэтому в выводах не требуем глубокого погружения в предмет и тщательного объяснения происходящего. Достаточно указать какие методы, на ваш взгляд, могут иметь потенциал при изучении данной темы, а какие стоит отбросить на начальном этапе.

Оформите всё красиво и структурированно.

*) Также можете реализовать на выбор какую-либо метрику качества кластеризации и выводить ее в процессе исследования. Поясните полученные результаты.

In [ ]:
# Стандартизация данных перед кластеризацией
X_scaled = StandardScaler().fit_transform(X_pca[:, :optimal_components])

In [ ]:
from sklearn.metrics import silhouette_score, silhouette_samples

def silhouette_visualize(X, cluster_labels, figsize):
    """Функция для визуализации силуэтов кластеров.

    Args:
        X: данные
        cluster_labels: метки кластеров
        figsize: размер изображения
    """
    silhouette_vals = silhouette_samples(X, cluster_labels)
    silhouette_avg = silhouette_score(X, cluster_labels)

    y_lower = 10
    unique_labels = np.unique(cluster_labels)
    plt.figure(figsize=figsize)

    # Для каждого кластера рисуем силуэты
    for i, label in enumerate(unique_labels):
        if label == -1:
            continue  # Пропускаем выбросы, если такие есть

        cluster_silhouette_vals = silhouette_vals[cluster_labels == label]
        cluster_silhouette_vals.sort()

        cluster_size = cluster_silhouette_vals.shape[0]
        y_upper = y_lower + cluster_size

        plt.fill_betweenx(np.arange(y_lower, y_upper),
                          0, cluster_silhouette_vals,
                          alpha=0.7)

        y_lower = y_upper + 10

    plt.axvline(silhouette_avg, color="red", linestyle="--")
    plt.title("Silhouette Analysis")
    plt.xlabel("Silhouette Coefficient")
    plt.ylabel("Cluster")
    plt.show()

**DBSCAN**

In [ ]:
def DBSCAN_visualize(eps, min_samples):
    """Функция для визуализации работы DBSCAN

    Args:
        eps и min_samples: гиперпараметры алгоритма
    """
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    labels = dbscan.fit_predict(X_scaled)

    # Визуализация результатов кластеризации
    draw_umap_tsne(figsize=(20, 7), c=labels)

    # Выводим силуэтный коэффициент, если кластеры обнаружены
    if len(np.unique(labels)) > 1:
        silhouette_visualize(X_scaled, labels, figsize=(10, 6))
        silhouette_avg = silhouette_score(X_scaled, labels)
        print(f"Silhouette Score for DBSCAN: {silhouette_avg}")
    else:
        print("No clusters found by DBSCAN.")

In [ ]:
DBSCAN_visualize(eps=2.5, min_samples=5)

**Вывод:**

**Параметры:** eps = 2.5, min_samples = 5.

**Silhouette Score:** 0.4076 — самый высокий среди всех алгоритмов, что указывает на лучшую плотность и однородность кластеров по сравнению с другими методами.

**Анализ результатов:** DBSCAN смог разделить данные на несколько плотных кластеров с высоким качеством по силуэтному коэффициенту. Это показывает, что DBSCAN хорошо справляется с задачей, когда данные имеют плотные группы и выбросы.

**Заключение:** DBSCAN демонстрирует наилучшие результаты среди всех методов в исходном пространстве данных и, вероятно, будет перспективен для дальнейших исследований. Однако параметр eps нужно точно настраивать для разных данных. А вот это очень муторно... Метод хороший, сам подбирает число кластеров, но вот подбор параметров - очень тонкий процесс.

**Agglomerative Clustering**

Для агломеративной кластеризации также постройте дендрограмму. Какого размера получаются кластеры?  Поясните по дендрограмме.

In [ ]:
from scipy.cluster.hierarchy import dendrogram

In [ ]:
def agglo_visualize(n_clusters):
    """Функция для визуализации работы агломеративной кластеризаии

    Args:
        n_clusters: количество кластеров
    """
    agglo = AgglomerativeClustering(n_clusters=n_clusters)
    labels = agglo.fit_predict(X_scaled)

    draw_umap_tsne(figsize=(20, 7), c=labels)
    silhouette_visualize(X_scaled, labels, figsize=(10, 6))
    silhouette_avg = silhouette_score(X_scaled, labels)
    print(f"Silhouette Score for Agglomerative Clustering: {silhouette_avg}")

    plt.figure(figsize=(10, 7))
    linkage_matrix = linkage(X_scaled, method='ward')
    dendrogram(linkage_matrix, truncate_mode='level', p=5)
    plt.title("Dendrogram for Agglomerative Clustering")
    plt.xlabel("Sample Index")
    plt.ylabel("Distance")
    plt.show()

In [ ]:
agglo_visualize(n_clusters=14)

**Вывод:**

**Параметры:** n_clusters = 14.

**Silhouette Score:** 0.0937 — самый низкий среди всех алгоритмов, что свидетельствует о слабой плотности и разделении кластеров.

**Анализ результатов:** Иерархическая кластеризация с фиксированным числом кластеров дала более размытые кластеры, что также видно на силуэтной диаграмме. Слабое разделение может быть связано с применением метода Ward, который минимизирует внутрикластерную дисперсию, но не всегда подходит для структур, в которых кластеры имеют разную плотность.

**Заключение:** Иерархическая кластеризация может быть менее эффективна на этом наборе данных. Возможно, ее стоит использовать в других представлениях данных или с иными методами связности. Также стоит учесть, то сам алгоритм работает очень долго, поэтому при большом объеме данных ему будет очень плохо.

**KMeans**

In [ ]:
def kmeans_visualize(n_clusters):
    """Функция для визуализации работы k-means

    Args:
        n_clusters: количество кластеров
    """
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    labels = kmeans.fit_predict(X_scaled)

    draw_umap_tsne(figsize=(20, 7), c=labels)
    silhouette_visualize(X_scaled, labels, figsize=(10, 6))
    silhouette_avg = silhouette_score(X_scaled, labels)
    print(f"Silhouette Score for KMeans: {silhouette_avg}")

In [ ]:
kmeans_visualize(n_clusters=14)

**Вывод:**

**Параметры:** n_clusters = 14.

**Silhouette Score:** 0.1782 — выше, чем у иерархической кластеризации, но значительно ниже, чем у DBSCAN.

**Анализ результатов:** Метод KMeans смог разбить данные на 14 кластеров с более высокими, но все же умеренными показателями силуэта. Вероятно, это связано с тем, что KMeans лучше подходит для данных с простыми сферическими кластерами и может не справиться с более сложной структурой данных.

**Заключение:** KMeans дал приемлемые результаты, но уступает DBSCAN. Он может быть полезен для грубого разбиения данных, но в случае сложных распределений его следует рассматривать осторожно. Но зато он работает быстрее всех.

Сделайте общие выводы.

DBSCAN показал себя лучшим методом для данной задачи, обеспечив максимальный силуэтный коэффициент, что указывает на наибольшую плотность и однородность кластеров. Агломеративная кластеризация и KMeans показали более низкие результаты по силуэту, и хотя KMeans смог создать четкие кластеры, его силуэтный коэффициент остается на среднем уровне. Иерархическая кластеризация показала слабые результаты и потребует дальнейших доработок для повышения качества кластеризации. Но с точки зрения учебной задачи, мы поняли, что параметры нужно настраивать очень долго, для достижения лучшего качества модели.